# Fit S(Q)

In [ ]:
# Chi-Huan Tung
# National Tsing-Hua University
# Aug 2021
#
# This notebook is based on the example of Convolutional Variational Autoencoder (CVAE)
# on tensorflow.org/tutorials/generative/cvae

## Setup

In [ ]:
from IPython import display

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import tensorflow_probability as tfp
import time
import scipy.interpolate as interp
tf.config.run_functions_eagerly(True)

Assign device (GPU)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Load data

### Test set

In [ ]:
# minimum sq
sq_min = np.exp(-5)

In [ ]:
if 1:
    X_file = '../data/input_grid_all_GPR80.csv'
    Y_file = '../data/target_grid_all.csv'
else:
    X_file = '../data/input_random_all_GPR80.csv'
    Y_file = '../data/target_random_all.csv'
    
fX_test = open(X_file, 'r', encoding='utf-8-sig')
sq_test = np.genfromtxt(fX_test, delimiter=',').astype(np.float32)
sq_test[sq_test<=0] = sq_min

fY_test = open(Y_file, 'r', encoding='utf-8-sig')
target_test = np.genfromtxt(fY_test, delimiter=',').astype(np.float32)

In [ ]:
sq_test.shape

In [ ]:
eta_test = target_test[:, 0]
kappa_test = target_test[:, 1]
Z_test = target_test[:, 3]
A_test = target_test[:, 2]
lnZ_test = np.log(Z_test)
lnA_test = np.log(A_test)

In [ ]:
sq_dim = sq_test.shape[1]
sample_test_dim = sq_test.shape[0]

$Q\cdot r_\textrm{ave}$

In [ ]:
q = (np.arange(sq_dim)+1)*0.2
q_rs = (np.arange(sq_dim)+1)*0.2
q_rs_dim = q_rs.shape[0]

Rescale

In [ ]:
r_eta_test = 1
sq_test_rs = np.zeros((sample_test_dim,q_rs_dim),dtype='float32')
for i in range(sample_test_dim):
    qr_eta = q*r_eta_test
    interpolating_function_test = interp.interp1d(qr_eta[3:],sq_test[i,3:],
                                                  fill_value='extrapolate',kind='linear')
    sq_test_rs[i,:] = interpolating_function_test(q_rs)
sq_test_rs[sq_test_rs<=0] = sq_min

### Mask

In [ ]:
mask_length = 0
sq_test_mask = sq_test_rs

for i in range(sample_test_dim):
    sq_test_mask[i,0:mask_length] = sq_test_mask[i,mask_length]

### Preprocess/Postprocess

In [ ]:
exp_scale = 3

def f_inp(sq):
    return np.log(sq)/exp_scale/2 + 0.5

def f_out(predictions):
    return np.exp((predictions*2-1)*exp_scale)

In [ ]:
def to_tf(arg):
    arg = tf.convert_to_tensor(arg, dtype=tf.float32)
    return arg

## Network architecture

In [ ]:
class VAE(tf.keras.Model):
    def __init__(self, latent_dim, sq_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        regularizer = None
        self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(sq_dim)),
            tf.keras.layers.Reshape((sq_dim,1)),
            tf.keras.layers.Conv1D(
                filters=32, kernel_size=3, strides=2, activation='relu',
                kernel_regularizer = regularizer,
                name='conv1d_en'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(
                latent_dim + latent_dim, 
                kernel_regularizer = regularizer,
                name='dense_en'),
        ]
        )
        
        self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(
                40*32, activation=tf.nn.relu, 
                kernel_regularizer = regularizer,
                name='dense_de'),
            tf.keras.layers.Reshape(target_shape=(40, 32)),
            tf.keras.layers.Conv1DTranspose(
                filters=32, kernel_size=3, strides=2, padding='same', activation='relu',
                kernel_regularizer = regularizer,
                name='conv1dtrs_de'),
            tf.keras.layers.Conv1DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same'),
            tf.keras.layers.Reshape((sq_dim,))
        ]
        )
        
    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(1000, self.latent_dim))
        return self.decode(eps, apply_sigmoid=True)
    
    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar
        
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean
    
    def decode(self, z, apply_sigmoid=False):
        logits = self.decoder(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits

In [ ]:
latent_dim = 3
model = VAE(latent_dim, q_rs_dim)

## Load and test model

In [ ]:
export_path = './saved_model/SQ_cVAE_MSE_ns/'
model_name = 'model_conv_stride2_GPR'
export_name = export_path + model_name

In [ ]:
reload_sm = model.load_weights(export_name, by_name=False, skip_mismatch=False, options=None)
reload_sm.__dict__

In [ ]:
model_r = reload_sm._root
#model_r.__dict__

### Loaded network architecture

In [ ]:
class VAE_r():
    def __init__(self):
        self.encoder = model_r.encoder
        self.decoder = model_r.decoder
        
    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = 0*tf.random.normal(shape=(1000, self.latent_dim))
        return self.decode(eps, apply_sigmoid=True)
    
    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar
        
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5)*0 + mean
    
    def decode(self, z, apply_sigmoid=False):
        logits = self.decoder(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits
    
M = VAE_r() # loaded model

## Define functions 

In [ ]:
def decoder(model,lv):
    z = model.reparameterize(lv, 0*lv)
    x = model.sample(z)
    
    return x

def encoder(model,x):
    mean = model.encode(x)[0]
    
    return mean

In [ ]:
def fit_loss(model, IQ_exp, fp):
    # form factor
    P = np.ones(IQ_exp.shape)
    
    # structure factor
    lv = tf.reshape(to_tf(fp[0:3]),(1,3))
    S = decoder(model,lv)
    
    # mean-square error
    IQ_th = P*S
    
    err = tf.reduce_mean((IQ_th-IQ_exp)**2)
    
    return err

## pick an I(Q) and test

In [ ]:
i_guess = (eta_test==0.03)&(kappa_test==0.2)&(A_test==10)
IQ_guess = f_inp(sq_test_mask[i_guess,:])
IQ_guess = to_tf(IQ_guess)
lv_guess = encoder(M,IQ_guess)

### initialize

In [ ]:
#sigma = tf.Variable(initial_value=1.0)
lv0 = tf.Variable(initial_value=0.0)
lv1 = tf.Variable(initial_value=0.0)
lv2 = tf.Variable(initial_value=0.0)
fp = [lv0,lv1,lv2]

In [ ]:
IQ_exp = f_inp(sq_test_mask[15000,:])
IQ_exp = to_tf(IQ_exp)

In [ ]:
epochs = 100
optimizer = tf.keras.optimizers.Adam(1e-1)
for epoch in range(1, epochs + 1):
    start_time = time.time()
    with tf.GradientTape() as tape:
        loss = fit_loss(M, IQ_exp, fp)
    gradients = tape.gradient(loss, fp)
    optimizer.apply_gradients(zip(gradients, fp))
    end_time = time.time()   
        
    lv_fit = tf.reshape(to_tf([lv0,lv1,lv2]),(1,3))
    S = decoder(M,lv_fit)[0,:]
    error = np.mean((f_out(IQ_exp)-f_out(S))**2)
    
    display.clear_output(wait=False)
    
    #print(error)
    plt.plot(f_out(IQ_exp),'k')
    plt.plot(f_out(S),'b')
    plt.text(60,0.375,'MSE = {:.5f}'.format(error))
    plt.text(60,0.25,'lv[0] = {:.2f}'.format(lv0.numpy()))
    plt.text(60,0.125,'lv[1] = {:.2f}'.format(lv1.numpy()))
    plt.text(60,0,'lv[2] = {:.2f}'.format(lv2.numpy()))
    
    pngname = './figures_SQ_cVAE_MSE_ns/fit_{:04d}.png'
    plt.savefig(pngname.format(epoch))
    plt.show()
    #time.sleep(0.1)

In [ ]:
gradients

In [ ]:
anim_file = './figures_SQ_cVAE_MSE_ns/Fit.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = glob.glob('./figures_SQ_cVAE_MSE_ns/fit_*.png')
    filenames = sorted(filenames)
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)